In [ ]:
using Pkg
if isfile("../Project.toml") && isfile("../Manifest.toml")
    Pkg.activate("..");
    ENV["PYTHON"] = "python3";
end

using Plots, Random, Distributions, LinearAlgebra
import Plots: Plot
import JuliaProbo: AbstractObject, Landmark, Map, KalmanFilter, EstimatorAgent, RealRobot, RealCamera, World, observations, push!, draw,decision, state_transition
gr();

In [ ]:
struct  Goal <: AbstractObject
    x::Float64
    y::Float64
    radius::Float64
    function Goal(x::Float64, y::Float64, radius = 0.3)
        new(x, y, radius)
    end
end

function draw(g::Goal, p::Plot{T}) where {T}
    p = plot!([g.x, g.x], [g.y, g.y+0.6], color="black", linewidth=2.0)
    p = plot!([g.x + 0.08], [g.y + 0.5], markershape=:rtriangle, markersize=9, color="red")
end

function ch10_puddle_world1()
    dt = 0.1
    # environment
    xlim = [-5.0, 5.0]
    ylim = [-5.0, 5.0]
    # id of landmark must start from 0 with 1 step
    landmarks =
        [Landmark([-4.0, 2.0], 0), Landmark([2.0, -3.0], 1), Landmark([3.0, 3.0], 2)]
    envmap = Map()
    push!(envmap, landmarks)
    world = World(xlim, ylim)
    push!(world, envmap)
    # robot side
    initial_pose = [0.0, 0.0, 0.0]
    estimator = KalmanFilter(envmap, initial_pose)
    agent = EstimatorAgent(0.2, 10.0 * pi / 180, dt, estimator)
    robot = RealRobot(initial_pose, agent, RealCamera(landmarks, distance_bias_rate_stddev = 0.0, direction_bias_stddev = 0.0); color = "red")
    push!(world, robot)
    goal = Goal(-3.0, -3.0)
    push!(world, goal)
    
    anim = @animate for i = 1:300
        t = dt * i
        annota = "t = $(round(t, sigdigits=3))[s]"
        p = draw(world, annota)
        z = observations(robot.sensor_, robot.pose_; noise = false, bias = false)
        v, ω = decision(agent, z, envmap)
        state_transition(robot, v, ω, dt; move_noise = true, vel_bias_noise = true)
    end
    gif(anim, "images/ch10_puddle_world1.gif", fps = 20)
end
ch10_puddle_world1()

<img src="images/ch10_puddle_world1.gif">